In [46]:
import os

# so I don't 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="-1"  #prevent access to GPU for inference

import pandas as pd
pd.options.display.max_rows = 150
pd.options.display.max_colwidth = 500
import json
from mdparse import transform_pre_rules, compose
from pathlib import Path
from tqdm import tqdm_notebook

from fastai.text import TextLMDataBunch as lmdb
from fastai.text.transform import Tokenizer
import pandas as pd
from pathlib import Path
from random import shuffle

from fastai.text.models import AWD_LSTM, awd_lstm_lm_config
from fastai.text import TextLMDataBunch as lmdb, load_data
from fastai.text.learner import language_model_learner
from fastai.basic_train import load_learner
from fastai.callbacks import EarlyStoppingCallback, SaveModelCallback, ReduceLROnPlateauCallback, CSVLogger

In [47]:
awd_lstm_lm_config

{'emb_sz': 800,
 'n_hid': 1800,
 'n_layers': 3,
 'pad_token': 1,
 'qrnn': False,
 'bidir': False,
 'output_p': 0.1,
 'hidden_p': 0.15,
 'input_p': 0.25,
 'embed_p': 0.02,
 'weight_p': 0.2,
 'tie_weights': True,
 'out_bias': True}

In [50]:
path = Path('lang_model_debug/')

def pass_through(x):
    return x


valid_df = pd.read_pickle('valid_df.pkl').dropna().drop_duplicates().head(25)
train_df = valid_df

In [51]:
tokenizer = Tokenizer(pre_rules=[pass_through], n_cpus=30)

data_lm = lmdb.from_df(path=path,
                       train_df=train_df,
                       valid_df=valid_df,
                       text_cols='text',
                       tokenizer=tokenizer)

In [52]:
learn = language_model_learner(data=data_lm,
                               arch=AWD_LSTM,
                               pretrained=False)

In [54]:
learn.path = Path('lang_model_debug_v2')

In [55]:
escb = EarlyStoppingCallback(learn=learn, patience=4)
smcb = SaveModelCallback(learn=learn)
rpcb = ReduceLROnPlateauCallback(learn=learn, patience=3)
csvcb = CSVLogger(learn=learn)
callbacks = [escb, smcb, rpcb, csvcb]

In [56]:
learn.fit(1, callbacks=callbacks)

epoch,train_loss,valid_loss,accuracy,time
0,6.153545,6.000374,0.127567,00:17


Better model found at epoch 0 with valid_loss value: 6.000373840332031.


In [16]:
type(learn.callbacks[0])

fastai.callbacks.rnn.RNNTrainer

In [18]:
! ls lang_model_debug/

history.csv  models


In [20]:
data_lm.save()

In [57]:
learn.bs

AttributeError: 'LanguageLearner' object has no attribute 'bs'

## Reload

In [41]:
awd_lstm_lm_config

{'emb_sz': 800,
 'n_hid': 1800,
 'n_layers': 3,
 'pad_token': 1,
 'qrnn': False,
 'bidir': False,
 'output_p': 0.1,
 'hidden_p': 0.15,
 'input_p': 0.25,
 'embed_p': 0.02,
 'weight_p': 0.2,
 'tie_weights': True,
 'out_bias': True}

In [62]:
data_lm = load_data(path, bs=128)

In [59]:
learn = language_model_learner(data=data_lm,
                               arch=AWD_LSTM,
                               pretrained=False,
                               config=awd_lstm_lm_config.update(dict(n_hid=1800, emb_sz=800)),
                              )

#learn.load('bestmodel')

In [40]:
learn.lr_find(num_it=1)

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [31]:
learn.summary()

Layer (type)         Output Shape         Param #    Trainable 
RNNDropout           [70, 800]            0          False     
______________________________________________________________________
RNNDropout           [70, 1800]           0          False     
______________________________________________________________________
RNNDropout           [70, 1800]           0          False     
______________________________________________________________________
Linear               [70, 1036]           829,836    True      
______________________________________________________________________
RNNDropout           [70, 800]            0          False     
______________________________________________________________________

Total params: 829,836
Total trainable params: 829,836
Total non-trainable params: 0

In [5]:
data_lm = load_data(path, bs=128)


learn = language_model_learner(data=data_lm,
                               arch=AWD_LSTM,
                               pretrained=False)

learn.load('bestmodel')
# callbacks
escb = EarlyStoppingCallback(learn=learn, patience=4)
smcb = SaveModelCallback(learn=learn)
rpcb = ReduceLROnPlateauCallback(learn=learn, patience=3)
csvcb = CSVLogger(learn=learn)
callbacks = [escb, smcb, rpcb, csvcb]


learn.fit(epochs=4, lr=1e-5, callbacks=callbacks)

epoch,train_loss,valid_loss,accuracy,time
0,6.690278,6.550413,0.262723,00:09
1,6.686635,6.532144,0.262723,00:09
2,6.679404,6.511220,0.262723,00:09
3,6.674406,6.489294,0.262723,00:09


Better model found at epoch 0 with valid_loss value: 6.550412654876709.
Better model found at epoch 1 with valid_loss value: 6.532143592834473.
Better model found at epoch 2 with valid_loss value: 6.5112199783325195.
Better model found at epoch 3 with valid_loss value: 6.489294052124023.


In [25]:
learn.export()

In [26]:
learn2 = load_learner(path)

In [30]:
! cat lang_model_onecycle/history.csv

epoch,train_loss,valid_loss,accuracy,time
0,3.726073,3.523209,0.375969,15:40:15
